# Notebook 08: Oxford RobotCar Integrated Validation (RQ2)

This notebook demonstrates **RQ2: Rare-Event & Real-World Validation**.
We use the **Oxford RobotCar Dataset** (via an adapter) to test if our Safety Metrics pipeline
can evaluate safety on real-world autonomous driving data.

**Mapping:**
- **Ego Vehicle** -> Robot
- **Pedestrians** -> Person Safety Objects

**Goal:** Compute SVR on a non-IID trajectory to validate the metric's utility beyond simulation.

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.append(os.path.abspath('../src'))

from data_loader.oxford_adapter import OxfordAdapter
from metrics.safety_evaluator import SafetyEvaluator

## 1. Load RobotCar Scenario
Use the adapter to load a snippet (or generate a mock proxy snippet matching the schema).

In [ ]:
adapter = OxfordAdapter()
objects, log_df = adapter.load_scenario(duration_s=60.0)

print(f"Loaded scenario with {len(objects)} objects and {len(log_df)} timesteps.")
print(log_df.head())

## 2. Evaluate Safety (Real-World SVR)
We apply the same `SafetyEvaluator` used in the Hospital Sim.
Note: Timestamps and distances are processed identically.

In [ ]:
evaluator = SafetyEvaluator(objects)
metrics, step_df = evaluator.evaluate_episode(log_df)

print("=== RobotCar Metric Validation ===")
print(f"SVR (Pedestrian): {metrics['SVR']*100:.2f}%")
print(f"Min Dist to Pedestrian: {metrics['Min_Dist_Person']:.2f} m")

## 3. Visualize Interaction
Plot the Ego Vehicle passing the Pedestrians.

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(log_df['x'], log_df['y'], label='Ego Trajectory')

# Plot Objects
for obj in objects:
    if obj['type'] == 'person':
        plt.plot(obj['pose']['x'], obj['pose']['y'], 'ro', label='Pedestrian')
        # Draw safety circle (0.5m crit)
        circle = plt.Circle((obj['pose']['x'], obj['pose']['y']), 0.5, color='r', fill=False)
        plt.gca().add_patch(circle)

plt.axis('equal')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.title('RobotCar Scenario: Ego vs Pedestrians')
plt.legend()
plt.grid(True)
plt.show()